<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Arthropod_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Arthropod Taxonomy Orders Object Detection Dataset](https://www.kaggle.com/datasets/mistag/arthropod-taxonomy-orders-object-detection-dataset/)



###### Lepidoptera = butterfies and moths
###### Hymenoptera = wasps, bees and ants
###### Hemiptera = true bugs (cicadas, aphids, shield bugs, ...)
###### Odonata = dragonflies
###### Diptera = fies
###### Araneae = spiders
###### Coleoptera = beetles

###### NOT IN DATASET
###### Orthoptera = grasshoppers

In [1]:
import os
import re
import sys
import json
import time
import numpy as np
import pprint as pp
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib as mpl
import concurrent.futures
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt


AUTO = tf.data.AUTOTUNE
pp = pp.PrettyPrinter()

!pip install --quiet tf-models-official==2.9.2
sys.path.append('PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER')# load all the metadata

import tensorflow_models as tfm
from tensorflow_models.vision import box_ops as box

     |████████████████████████████████| 2.1 MB 8.9 MB/s 
     |████████████████████████████████| 662 kB 51.9 MB/s 
     |████████████████████████████████| 1.1 MB 57.5 MB/s 
     |████████████████████████████████| 118 kB 62.0 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 238 kB 11.3 MB/s 
     |████████████████████████████████| 352 kB 48.5 MB/s 
     |████████████████████████████████| 1.3 MB 51.5 MB/s 
     |████████████████████████████████| 4.6 MB 52.9 MB/s 


In [2]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/'
!kaggle datasets download -d mistag/arthropod-taxonomy-orders-object-detection-dataset
!unzip *.zip && rm *.zip

Streaming output truncated to the last 5000 lines.
  inflating: ArTaxOr/Lepidoptera/c9b07301ed0e.jpg  
  inflating: ArTaxOr/Lepidoptera/c9d742746b8a.jpg  
  inflating: ArTaxOr/Lepidoptera/c9d78061b77d.jpg  
  inflating: ArTaxOr/Lepidoptera/ca2ff8921244.jpg  
  inflating: ArTaxOr/Lepidoptera/ca338092c0ec.jpg  
  inflating: ArTaxOr/Lepidoptera/ca35e5c89d94.jpg  
  inflating: ArTaxOr/Lepidoptera/ca4cb66fcd4a.jpg  
  inflating: ArTaxOr/Lepidoptera/ca5e1fe4cbc0.jpg  
  inflating: ArTaxOr/Lepidoptera/ca5e2a4139c0.jpg  
  inflating: ArTaxOr/Lepidoptera/ca63af0a9774.jpg  
  inflating: ArTaxOr/Lepidoptera/ca652f69b2e7.jpg  
  inflating: ArTaxOr/Lepidoptera/ca95914f205e.jpg  
  inflating: ArTaxOr/Lepidoptera/ca997241704b.jpg  
  inflating: ArTaxOr/Lepidoptera/cb0ad7fb32fa.jpg  
  inflating: ArTaxOr/Lepidoptera/cb0aec5a6b79.jpg  
  inflating: ArTaxOr/Lepidoptera/cb134256077c.jpg  
  inflating: ArTaxOr/Lepidoptera/cb22cb2317b4.jpg  
  inflating: ArTaxOr/Lepidoptera/cb35eb6c2e67.jpg  
  inflating: 

# [Data Exploring](https://www.kaggle.com/code/mistag/starter-arthropod-taxonomy-orders-data-exploring)

The Arthropod Taxonomy Orders dataset is a collection of highres images annotated with labels from the taxanomy rank order. Annotations have been made with VoTT. VoTT stores all metadata in json files. In this kernel we will import all the metadata into DataFrames and export metadata to a variety of formats for object detection model training.
The dataset is distributed under CC BY-NC-SA 4.0

In [3]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import glob

pfiles = glob.glob('/content/ArTaxOr/**/*.vott', recursive=True)

df = pd.DataFrame()
for p in pfiles:
  print(p)
  with open(p) as f:
    pdata = json.load(f)
    print(pdata)
    df = df.append(pd.DataFrame(list(pdata['assets'].values())), ignore_index=True)

df['path'] = df['path'].str.replace('file:F:/', '')

/content/ArTaxOr/Lepidoptera/annotations/Lepidoptera.vott


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
df

{'name': 'Odonata', 'securityToken': 'Diptera Token', 'sourceConnection': {'name': 'Odonata', 'providerType': 'localFileSystemProxy', 'providerOptions': {'encrypted': 'eyJjaXBoZXJ0ZXh0IjoiODY0NWJmNDA1NDNjOGVlYmZhZDQzZTliYzU4NjM0MjliZDcyNjRlODJhYzRkODNhNGFhMzhiY2U2N2UyYmFjOTU3MmY4NWEzYWVkYjI0ODAwODU3MGExOWUyMzRmZWQ4IiwiaXYiOiI1ZDkzYjc2MWI5ZTg2MzUxOWVlYzlkYmIzZjNkMTA3ODUyZmIyOTlmY2ZmZWExYWMifQ=='}, 'id': 'Ko1s85UOf'}, 'targetConnection': {'name': 'Odonata annotations', 'providerType': 'localFileSystemProxy', 'providerOptions': {'encrypted': 'eyJjaXBoZXJ0ZXh0IjoiOGVkNTIwMDU5OGM0NWE1MDY2MzgwMTE1MTkxMGMyNDcxNjI4ZWRjOGQ4M2EwMzQ3OTU0NmU3NmE0MDY4MTZmNGNhNDAzODFhMjliNWJlODM3NzQ0MjEyOTdjZjc4NmNhODhhZWQzNDdhZGRiZmZiNTIwODY2MDRmN2Y0ZTczNTUiLCJpdiI6IjUyYTgzMTBlOWRlNWViNjVjMzU5ODFiYWFjYmMxYWMwYzE3NzMzMGQ1MTU2MmNjMyJ9'}, 'id': 'csXqIsO36'}, 'videoSettings': {'frameExtractionRate': 15}, 'tags': [{'name': 'Lepidoptera', 'color': '#5db300'}, {'name': 'Coleoptera', 'color': '#e81123'}, {'name': 'Hymenopt

,format,id,name,path,size,state,type
0,jpg,9c8f23b82ee4a2d682291faa7a56cbbf,00026d79d067.jpg,ArTaxOr/Lepidoptera/00026d79d067.jpg,"{'width': 3888, 'height': 2592}",2,1
1,jpg,9047380761065400114098ba4a37eb4c,00028c56bf09.jpg,ArTaxOr/Lepidoptera/00028c56bf09.jpg,"{'width': 5184, 'height': 3456}",2,1
2,jpg,511bf35dd66f49103ff9ee830d6022e0,00126d9a3bea.jpg,ArTaxOr/Lepidoptera/00126d9a3bea.jpg,"{'width': 4032, 'height': 3024}",2,1
3,jpg,518d6bc681b944adb5fbfecb2e78ac13,002b37ac08e1.jpg,ArTaxOr/Lepidoptera/002b37ac08e1.jpg,"{'width': 1920, 'height': 1521}",2,1
4,jpg,4338bc92a1392d3d5a61ed0c90a56be4,0032708c2542.jpg,ArTaxOr/Lepidoptera/0032708c2542.jpg,"{'width': 4608, 'height': 3456}",2,1
...,...,...,...,...,...,...,...
15371,jpg,27c45505c9d0f024745b6cad9b232d6b,ff926865268a.jpg,ArTaxOr/Odonata/ff926865268a.jpg,"{'width': 2000, 'height': 1537}",2,1
15372,jpg,c6e4c8624ca87afdfd590f04ebcc6871,ffab00fe2499.jpg,ArTaxOr/Odonata/ffab00fe2499.jpg,"{'width': 2048, 'height': 1329}",2,1
15373,jpg,25569532ac3f65cd68c8ad53ceaead14,ffc1fd21a8bb.jpg,ArTaxOr/Odonata/ffc1fd21a8bb.jpg,"{'width': 2048, 'height': 1407}",2,1
15374,jpg,ff13dae44aead408c5a11f6063e29075,ffc4adb8b04e.jpg,ArTaxOr/Odonata/ffc4adb8b04e.jpg,"{'width': 1384, 'height': 1038}",2,1


In [6]:
labels = pd.DataFrame(list(pdata['tags']))[:-3]
labels

,name,color
0,Lepidoptera,#5db300
1,Coleoptera,#e81123
2,Hymenoptera,#6917aa
3,Diptera,#015cda
4,Araneae,#4894fe
5,Hemiptera,#257ffe
6,Odonata,#257ffe


In [7]:
p = 'ArTaxOr/Diptera/fff86b0ae807.jpg'
index = p[::-1].find('/')+1
p[:-index]

'ArTaxOr/Diptera'

In [8]:
anno=pd.DataFrame(columns=['label', 'label_idx', 'xres', 'yres', 'height', 'width', 'left', 'top', 
                           'right', 'bottom', 'area', 'xcenter', 'ycenter', 'blurred',
                           'occluded', 'truncated', 'file', 'id'])


#get all the image's annotation details (object data) from a json file and store it in a dataframe
for i in range(len(df)):  #15376 images(with details)
  p = df['path'][i]
  index = p[::-1].find('/')+1
  json_file=f'/content/{p[:-index]}/annotations/{df["id"][i]}-asset.json'

  if os.path.isfile(json_file):
    with open(json_file) as f:
      adata = json.load(f)
    xres=adata['asset']['size']['width']
    yres=adata['asset']['size']['height'] 
    for j in range(len(adata['regions'])):
            h=adata['regions'][j]['boundingBox']['height']/yres
            w=adata['regions'][j]['boundingBox']['width']/xres
            tags=adata['regions'][j]['tags']
            anno=anno.append({'label': tags[0],
                              'label_idx': labels[labels.name==tags[0]].index[0],
                              'xres': xres,
                              'yres': yres,
                              'height': h,
                              'width': w,                              
                              'left': adata['regions'][j]['boundingBox']['left']/xres,
                              'top': adata['regions'][j]['boundingBox']['top']/yres,
                              'right': adata['regions'][j]['boundingBox']['left']/xres+w,
                              'bottom': adata['regions'][j]['boundingBox']['top']/yres+h, 
                              'area': h*w,
                              'xcenter': adata['regions'][j]['boundingBox']['left']/xres+0.5*w,
                              'ycenter': adata['regions'][j]['boundingBox']['top']/yres+0.5*h,
                              'blurred': int(any(ele == '_blurred' for ele in tags)),
                              'occluded': int(any(ele == '_occluded' for ele in tags)),
                              'truncated': int(any(ele == '_truncated' for ele in tags)),
                              'file': adata['asset']['path'].replace('file:F:/',''),
                              'id': adata['asset']['id'],}, ignore_index=True)

In [9]:
anno.sample(5)

,label,label_idx,xres,yres,height,width,left,top,right,bottom,area,xcenter,ycenter,blurred,occluded,truncated,file,id
9415,Diptera,3,2220,2112,0.035441,0.036463,0.300094,0.097701,0.336557,0.133142,0.001292,0.318326,0.115421,1,1,0,ArTaxOr/Diptera/3a7e74d3f468.jpg,cb08f9c01bc3f77a71dd258284cfdede
7391,Coleoptera,1,2048,1492,0.942529,0.946245,0.053755,0.000000,1.000000,0.942529,0.891863,0.526878,0.471264,0,0,1,ArTaxOr/Coleoptera/765c2fefd3c8.jpg,fa92754c661861a3853887b53e426f86
11069,Diptera,3,1972,1793,0.706897,0.856272,0.065875,0.211686,0.922147,0.918582,0.605296,0.494011,0.565134,0,0,0,ArTaxOr/Diptera/e6422ed2b71e.jpg,df0a3987594488e4d828612a7d2a8279
15394,Diptera,3,4032,3024,0.273946,0.249282,0.082256,0.380268,0.331537,0.654215,0.068290,0.206897,0.517241,0,0,0,ArTaxOr/Hymenoptera/7335fcc40b82.jpg,ca22ec872230ea0a78b1b2f9478e77f1
7170,Coleoptera,1,2048,1599,0.650383,0.683620,0.133192,0.224138,0.816812,0.874521,0.444615,0.475002,0.549330,0,0,0,ArTaxOr/Coleoptera/5dea2281084f.jpg,d7e120395b37932520015737ce6de467


In [10]:
# sns.relplot(x="width", y="height", hue="label", col="label", data=anno)

In [11]:
# sns.jointplot(x='width', y='height', data=anno.loc[anno['label'] == 'Diptera'])

In [12]:
sns.set(rc={'figure.figsize': (12,6)})
# sns.violinplot(x=anno['label'], y=anno['area'])

In [13]:
# graph=sns.countplot(data=anno, x='label')
# graph.set_xticklabels(graph.get_xticklabels(), rotation=90)

# for p in graph.patches:
#   height = p.get_height()
#   graph.text(p.get_x() + p.get_width()/2, height + 0.1, height, ha='center')

In [14]:
temp = 'truncated'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
# sns.countplot(x=temp, hue='label', data=df2)

In [15]:
temp = 'blurred'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
# sns.countplot(x=temp, hue='label', data=df2)

In [16]:
temp = 'occluded'
df2 = anno[['label', temp]]
df2 = df2.loc[df2[temp] == 1]
# sns.countplot(x=temp, hue='label', data=df2)

## Image exploration


In [17]:
def attribution(fname):
    img = Image.open(fname)
    exif_data = img._getexif()
    img.close()
    if len(exif_data[315]) > 0:
        s='Photo: '+exif_data[315]
    else:
        s=exif_data[37510][8:].decode('ascii')
    return s

def plot_img(axes, image_df, highlight=True):
    image_file_name='/content/'+image_df.iloc[0].file
    im = Image.open(image_file_name)
    im.thumbnail((300,300),Image.ANTIALIAS)  #A small image representation of a larger image
    draw = ImageDraw.Draw(im)
    xres, yres = im.size[0], im.size[1] # X, Y Resolution
    for i in range(len(image_df)):
        selected_img = image_df.iloc[i]
        if highlight==True:
            color=(255, 0, 0) if i == 0 else (128, 128, 128)          
        else:
            color=labels[labels.name == selected_img.label].color.iloc[0]
        
        #draw a boundingbox
        draw.rectangle([int(selected_img['left']*xres),
                        int(selected_img['top']*yres),
                        int(selected_img['right']*xres),
                        int(selected_img['bottom']*yres)], outline=color, width=2)
        
    plt.setp(axes, xticks=[], yticks=[])  #remove the grids
    axes.set_title(image_df.iloc[0].label+'\n'+attribution(image_file_name))
    plt.imshow(im)

In [42]:
fig = plt.figure(figsize=(16,26))
for i in range(len(labels)):
  for j in range(3):
    #Select 3 images with the largest area
    # ldf=anno[anno.label == labels.name[i]].nlargest(3, 'area') 

    #Select images with the most objects
    a=anno[anno.label == labels.name[i]]['id'].value_counts()
    ldf=anno[anno.id == a.index[j]]
    # axes = fig.add_subplot(len(labels), 3, 1+i*3+j)
    # plot_img(axes, anno[anno.id == ldf.iloc[j].id].sort_values(by=['area'], ascending=False), highlight=False)

<Figure size 1152x1872 with 0 Axes>

## Metadata export

export metadata to various formats for object detection model training


### CSV

In [19]:
header = ['file', 'label', 'height', 'width', 'left', 'top', 'right', 'bottom'] # change as required
anno.to_csv('./ArTaxOr.csv', index=False, columns = header) 

### Pascal VOC

Pascal VOC files are xml format, and there is one xml file per image file, with same name.

In [20]:
!{sys.executable} -m pip install pascal_voc_writer
from pascal_voc_writer import Writer

if not os.path.exists('voc'):
    os.mkdir('voc')

for i in range(len(df)):
    ldf=anno[anno.id == df.id[i]].reset_index()
    p=df.path[i].split('/')
    image_xml = p[2].replace('.jpg','.xml') 
    width, height = ldf.xres[0], ldf.yres[0]
    
    writer = Writer(df.path[i], width, height)
    for j in range(len(ldf)):
        writer.addObject(ldf.label[j], 
                         int(ldf.left[j]*width), 
                         int(ldf.top[j]*height), 
                         int(ldf.right[j]*width),
                         int(ldf.bottom[j]*height))
    writer.save(f'./voc/{image_xml}')
# print(os.listdir("./voc"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Darknet YOLOv3
Darknet expects one annotation file per image file. Each object is described by:

`class x_center y_center width height`

In [21]:
if not os.path.exists('labels'):
    os.mkdir('labels')

for i in range(len(df)):
    ldf=anno[anno.id == df.id[i]].reset_index()
    p=df.path[i].split('/') 
    image_txt = p[2].replace('.jpg','.txt') 
    
    file=open(f'./labels/{image_txt}','w')
    for j in range(len(ldf)):
        label_name=labels[labels.name == ldf.label[j]].index.to_list()
        file.write(f'{label_name[0]} {ldf.xcenter[j]} {ldf.ycenter[j]} {ldf.width[j]} {ldf.height[j]}\n')
    file.close()
# print(os.listdir("./labels"))

### Pickle

In [22]:
labels.to_pickle('./ArTaxOr_labels.pkl')
df.to_pickle('./ArTaxOr_filelist.pkl')
anno.to_pickle('./ArTaxOr_objects.pkl')

### TensorFlow TFRecords

#### Protocol buffers
TFRecords are Protocol buffers. Protocol buffers are a way of serializing structured data in a compact and efficient way. A Protocol buffer message is defined by a .proto file. Example:

```
syntax = "proto3"; // define protocol version

// Define "Employee" message with 3 fields
message Employee {
 string name = 1; // a unique number is assigned to each field
 int32 company_id = 2;
 string address = 3;
}

```


A single .proto-file can define multiple messages and messages can be nested. The beauty of protocol buffers is that the .proto-files can be compiled into a library (language of choice) with access functions using the protoc compiler. But for TFRecords the access functions we need are already part of TensorFlow, so no need to worry about compiling .proto files.

#### TFRecord format with simple data
TFRecords are made to support just about any type of data, and that means nesting basic features into a hierarchy of features.TFRecords supports only three different data types:

bytes
float
int64

So any data must be converted into one of these three types.

In [23]:
# tf.train.Feature can be used to convert data into lists of these types.
int_list = tf.train.Feature(int64_list=tf.train.Int64List(value=np.arange(8)))
int_list

int64_list {
  value: 0
  value: 1
  value: 2
  value: 3
  value: 4
  value: 5
  value: 6
  value: 7
}

In [24]:
float_list = tf.train.Feature(float_list=tf.train.FloatList(value=np.random.rand(10)))
float_list

float_list {
  value: 0.599125325679779
  value: 0.3095448911190033
  value: 0.221328005194664
  value: 0.14357033371925354
  value: 0.48036739230155945
  value: 0.8367927074432373
  value: 0.10189275443553925
  value: 0.0751216858625412
  value: 0.6496613621711731
  value: 0.36178693175315857
}

In [25]:
bytes_list = tf.train.Feature(bytes_list=tf.train.BytesList(value=['hello, my name is Esmail'.encode()]))
bytes_list

bytes_list {
  value: "hello, my name is Esmail"
}

In [26]:
features = tf.train.Features(feature={
    'integers': int_list,
    'float': float_list,
    'description': bytes_list
})
features

feature {
  key: "description"
  value {
    bytes_list {
      value: "hello, my name is Esmail"
    }
  }
}
feature {
  key: "float"
  value {
    float_list {
      value: 0.599125325679779
      value: 0.3095448911190033
      value: 0.221328005194664
      value: 0.14357033371925354
      value: 0.48036739230155945
      value: 0.8367927074432373
      value: 0.10189275443553925
      value: 0.0751216858625412
      value: 0.6496613621711731
      value: 0.36178693175315857
    }
  }
}
feature {
  key: "integers"
  value {
    int64_list {
      value: 0
      value: 1
      value: 2
      value: 3
      value: 4
      value: 5
      value: 6
      value: 7
    }
  }
}

The last step is to create an example TFRecord. So, example is not a very good name, but that is what the TFRecord is called. We simply pass the list of features to `tf.train.Example.`

In [27]:
tf_record = tf.train.Example(features=features)
tf_record

features {
  feature {
    key: "description"
    value {
      bytes_list {
        value: "hello, my name is Esmail"
      }
    }
  }
  feature {
    key: "float"
    value {
      float_list {
        value: 0.599125325679779
        value: 0.3095448911190033
        value: 0.221328005194664
        value: 0.14357033371925354
        value: 0.48036739230155945
        value: 0.8367927074432373
        value: 0.10189275443553925
        value: 0.0751216858625412
        value: 0.6496613621711731
        value: 0.36178693175315857
      }
    }
  }
  feature {
    key: "integers"
    value {
      int64_list {
        value: 0
        value: 1
        value: 2
        value: 3
        value: 4
        value: 5
        value: 6
        value: 7
      }
    }
  }
}

 write this record to a file using tf.io.TFRecordWriter

In [28]:
fname='example1.tfrecord'
with tf.io.TFRecordWriter(fname) as writer:
    writer.write(tf_record.SerializeToString())
print(f"Size of {fname} is {os.path.getsize(fname)} bytes")

Size of example1.tfrecord is 144 bytes


Next, read the file back in and access the data using tf.data.TFRecordDataset and tf.train.Example.FromString:

In [29]:
dataset = tf.data.TFRecordDataset(fname)
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())
parsed

features {
  feature {
    key: "description"
    value {
      bytes_list {
        value: "hello, my name is Esmail"
      }
    }
  }
  feature {
    key: "float"
    value {
      float_list {
        value: 0.599125325679779
        value: 0.3095448911190033
        value: 0.221328005194664
        value: 0.14357033371925354
        value: 0.48036739230155945
        value: 0.8367927074432373
        value: 0.10189275443553925
        value: 0.0751216858625412
        value: 0.6496613621711731
        value: 0.36178693175315857
      }
    }
  }
  feature {
    key: "integers"
    value {
      int64_list {
        value: 0
        value: 1
        value: 2
        value: 3
        value: 4
        value: 5
        value: 6
        value: 7
      }
    }
  }
}

Access each feature using `parsed.features.feature['<feature name>']`



In [30]:
parsed.features.feature['description'].bytes_list.value[0].decode()

'hello, my name is Esmail'

In [31]:
parsed.features.feature['integers'].int64_list.value[:]

[0, 1, 2, 3, 4, 5, 6, 7]

In [32]:
parsed.features.feature['float'].float_list.value[:]

[0.599125325679779,
 0.3095448911190033,
 0.221328005194664,
 0.14357033371925354,
 0.48036739230155945,
 0.8367927074432373,
 0.10189275443553925,
 0.0751216858625412,
 0.6496613621711731,
 0.36178693175315857]

####TFRecords with image data

To make input function efficient, all preprocessing steps should be performed when creating TFRecords, otherwise the input function must repeat preprocessing each time the training passed over the TFRecord. Typical preprocessing steps are:

##### Decode image from jpg etc.
##### Resize image
##### Convert to float
##### Normalize to [0,1] range

Augumentation is not something to do in a TFRecord, since agumentation could be different each time training passes over the data

In [33]:
import hashlib
from io import BytesIO
from PIL import Image, ImageFont, ImageDraw
ARTAXOR_PATH = '/content/'

pickles='/content/'
objectdf=pd.read_pickle(pickles+'ArTaxOr_objects.pkl')
labels=pd.read_pickle(pickles+'ArTaxOr_labels.pkl')
objectdf.sample(5)

,label,label_idx,xres,yres,height,width,left,top,right,bottom,area,xcenter,ycenter,blurred,occluded,truncated,file,id
7792,Coleoptera,1,1500,1000,0.783868,0.978635,0.021365,0.121688,1.000000,0.905555,0.767121,0.510682,0.513621,0,0,1,ArTaxOr/Coleoptera/9e52f6e405be.jpg,4f126c22969e2ef5bccadc3222c5dbc9
5702,Hemiptera,5,4272,2848,0.041365,0.051724,0.337241,0.568091,0.388966,0.609456,0.002140,0.363103,0.588773,0,0,0,ArTaxOr/Hemiptera/d7285107eca0.jpg,bcd3c5c2a465acd368de6bdca687c7eb
14380,Hymenoptera,2,2048,1536,0.228927,0.113506,0.483836,0.394636,0.597342,0.623563,0.025985,0.540589,0.509100,0,0,0,ArTaxOr/Hymenoptera/2048af204758.jpg,ff7e63d39794f36a94eea6019979059c
19277,Odonata,6,1785,2048,0.705607,0.690404,0.000000,0.293614,0.690404,0.999221,0.487154,0.345202,0.646417,0,1,1,ArTaxOr/Odonata/d87295399cea.jpg,0ef47fa5dd44e85c20b53d47da18fe17
7464,Coleoptera,1,1442,1082,0.758621,0.636233,0.153262,0.123563,0.789495,0.882184,0.482659,0.471379,0.502874,0,0,0,ArTaxOr/Coleoptera/7df01393da0e.jpg,251a197906ff8ceaadc30caf82493bb5


Define a function that creates a tf.train.Example from an image and the objects contained within.

    Note that TensorFlow Object Detection API expects jpeg-encoded image data

, so the only preprocessing to be done is resize (optional) before the image is re-encoded (via a memory buffer using BytesIO).


The input to this function is a DataFrame that contains one row per object and the columns shown above. This record is generous with features, but that is to make sure that the TF Object Detection API will successfully run the evaluation phase during training.

In [34]:
# Fetch attribution string from image EXIF data
def get_attribution(file):
    with Image.open(file) as img:
        exif_data = img._getexif()
    
    source ='Photo: unknown'
    if exif_data is not None:
        if 37510 in exif_data and len(exif_data[37510]) > 0:
            source = exif_data[37510][8:].decode('ascii')
        if 315 in exif_data and len(exif_data[315]) > 0:
            source = 'Photo: ' + exif_data[315]
    return source

# Create example for TensorFlow Object Detection API
def create_tf_example(imagedf, longest_edge=1024):  
    fname = ARTAXOR_PATH+imagedf.file.iloc[0]  #exclude image path 
    filename=fname.split('/')[-1]              #exclude image name
    by = get_attribution(fname)                #exclude EXIF data
    img = Image.open(fname, "r")

    # resize image if larger that longest edge while keeping aspect ratio
    if max(img.size) > longest_edge:
        img.thumbnail((longest_edge, longest_edge), Image.ANTIALIAS)
    
    width, height = img.size #(w, h)

    buf= BytesIO()
    img.save(buf, format= 'JPEG') # encode to jpeg in memory
    encoded_image_data= buf.getvalue()
    image_format = b'jpeg'
    
    source_id = filename.split('.')[0]
    license = 'CC BY-NC-SA 4.0'
    # A hash of the image is used in some frameworks
    key = hashlib.sha256(encoded_image_data).hexdigest() 

    # object bounding boxes 
    # List of normalized coordinates in bounding box (1 per box)
    xmins, xmaxs, ymins, ymaxs = imagedf[['left', 'right', 'top', 'bottom']].values.T

    # List of string class name & id of bounding box (1 per box)
    object_count = len(imagedf)
    classes_text = []
    classes = []
    for i in range(object_count):
        classes_text.append(imagedf.label.iloc[i].encode())
        classes.append(1 + imagedf.label_idx.iloc[i])

    #??????????????????????????????????????????????????????????????????
    
    # unused features from Open Image 
    depiction = np.zeros(object_count, dtype=int)
    group_of = np.zeros(object_count, dtype=int)
    occluded = imagedf.occluded.values #also Pascal VOC
    truncated = imagedf.truncated.values # also Pascal VOC


    # Pascal VOC
    view_text = []
    for i in range(object_count):
        view_text.append('frontal'.encode())
    difficult = np.zeros(object_count, dtype=int)

    tf_record = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[source_id.encode()])),
        'image/license': tf.train.Feature(bytes_list=tf.train.BytesList(value=[license.encode()])),
        'image/by': tf.train.Feature(bytes_list=tf.train.BytesList(value=[by.encode()])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image_data])),
        'image/key/sha256': tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode()])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_format])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
        'image/object/depiction': tf.train.Feature(int64_list=tf.train.Int64List(value=depiction)),
        'image/object/group_of': tf.train.Feature(int64_list=tf.train.Int64List(value=group_of)),
        'image/object/occluded': tf.train.Feature(int64_list=tf.train.Int64List(value=occluded)),
        'image/object/truncated': tf.train.Feature(int64_list=tf.train.Int64List(value=truncated)),
        'image/object/difficult': tf.train.Feature(int64_list=tf.train.Int64List(value=difficult)),
        'image/object/view': tf.train.Feature(bytes_list=tf.train.BytesList(value=view_text))
    }))
    return tf_record

In [35]:
#An Example of creating tf record
sample_file='ArTaxOr/Hemiptera/00600a41bcaf.jpg'
imagedf=objectdf[objectdf.file == sample_file]
tfr=create_tf_example(imagedf)
# print(tfr)
fname='./image_ex1.tfrecord'
with tf.io.TFRecordWriter(fname) as writer:
    writer.write(tfr.SerializeToString())
print(f"Size of {fname} is {fname, os.path.getsize(fname)//1024} kbytes")

Size of ./image_ex1.tfrecord is ('./image_ex1.tfrecord', 95) kbytes


load back in and visualize the Tensordlow records contents.

In [36]:
font = ImageFont.load_default()
def bbox(img, xmin, ymin, xmax, ymax, color, width, label, score):
    draw = ImageDraw.Draw(img)

    xres, yres = img.size  #(w, h)

    #box[0] = xmin*xres
    box = np.multiply([xmin, ymin, xmax, ymax], [xres, yres, xres, yres]).astype(int).tolist()

    txt = f" {label}: {round(score, 1)}" if score >= 0. else f" {label}"
    ts = draw.textsize(txt, font=ImageFont.load_default())
    draw.rectangle(box, outline=color, width=width)

    #######????????    
    if len(label) > 0:
        if box[1] >= ts[1]+3:
            xsmin, ysmin = box[0],                box[1]-ts[1]-3
            xsmax, ysmax = box[0] + ts[0] + 2,    box[1]
        else:
            xsmin, ysmin = box[0],               box[3]
            xsmax, ysmax = box[0] + ts[0] + 2,   box[3] + ts[1] + 1
            
        draw.rectangle([xsmin, ysmin, xsmax, ysmax], fill=color)
        draw.text((xsmin, ysmin), txt, font=font, fill='white')

In [37]:
def plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label, by):
    for i in range(len(xmin)):
        color=labels.color[class_label[i]-1]
        bbox(img, xmin[i], ymin[i], xmax[i], ymax[i], color, 5, classes[i].decode(), -1)
    plt.setp(axes, xticks=[], yticks=[])
    axes.set_title(by)
    plt.imshow(img)

In [38]:
# load tfrecord
fname='image_ex1.tfrecord'
dataset = tf.data.TFRecordDataset(fname)
img_example = next(iter(dataset))
img_parsed = tf.train.Example.FromString(img_example.numpy())
# only extract features we will actually use
xmin=img_parsed.features.feature['image/object/bbox/xmin'].float_list.value[:]
xmax=img_parsed.features.feature['image/object/bbox/xmax'].float_list.value[:]
ymin=img_parsed.features.feature['image/object/bbox/ymin'].float_list.value[:]
ymax=img_parsed.features.feature['image/object/bbox/ymax'].float_list.value[:]
by=img_parsed.features.feature['image/by'].bytes_list.value[0].decode()
classes=img_parsed.features.feature['image/object/class/text'].bytes_list.value[:]
class_label=img_parsed.features.feature['image/object/class/label'].int64_list.value[:]
img_encoded=img_parsed.features.feature['image/encoded'].bytes_list.value[0]

In [ ]:
fig = plt.figure(figsize=(10,10))
axes = axes = fig.add_subplot(1, 1, 1)
img = Image.open(BytesIO(img_encoded))
plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label, by)

Notice: putting jpeg-encoded images in TFRecords does not release the full potential of TFRecords. If you are training on a machine with fast I/O and lots of disk space, it is better to do all the preprocessing before storing the image as normalized float array. Let's define another function for this:

In [48]:
def create_tf_example2(imagedf, longest_edge=1024):  
    # Filename of the image (full path is useful when there are multiple image directories)
    fname = ARTAXOR_PATH+imagedf.file.iloc[0]  #exclude image path 
    filename=fname.split('/')[-1]              #exclude image name
    by = get_attribution(fname)                #exclude EXIF data
    img = Image.open(fname, "r")
    source_id = filename.split('.')[0]

    # resize image if larger that longest edge while keeping aspect ratio
    if max(img.size) > longest_edge:
        img.thumbnail((longest_edge, longest_edge), Image.ANTIALIAS)

    image_data = np.asarray(img)

    # storing shape will make it easy to reconstruct image later
    image_shape = np.array(image_data.shape)
    # convert to float
    image_data = image_data.reshape(image_data.shape[0] * image_data.shape[1] * image_data.shape[2])
    image_data = image_data.astype(float)/255. # normalize to [0,1]

    # object bounding boxes 
    # List of normalized coordinates in bounding box (1 per box)
    xmins, xmaxs, ymins, ymaxs = imagedf[['left', 'right', 'top', 'bottom']].values.T

    # List of string class name & id of bounding box (1 per box)
    classes_text = []
    classes = []
    for i in range(len(imagedf)):
        classes_text.append(imagedf.label.iloc[i].encode())
        classes.append(1+imagedf.label_idx.iloc[i])

    tf_record = tf.train.Example(features=tf.train.Features(feature={
        'image/shape': tf.train.Feature(int64_list=tf.train.Int64List(value=image_shape)),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[source_id.encode()])),
        'image/by': tf.train.Feature(bytes_list=tf.train.BytesList(value=[by.encode()])),
        'image/data': tf.train.Feature(float_list=tf.train.FloatList(value=image_data)),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes))
    }))
    return tf_record

In [51]:
sample_file2='ArTaxOr/Hymenoptera/ab30b4c2f70c.jpg'
imagedf=objectdf[objectdf.file == sample_file2]
tfr2 = create_tf_example2(imagedf)
fname2 = 'image_ex2.tfrecord'
with tf.io.TFRecordWriter(fname2) as writer:
    writer.write(tfr2.SerializeToString())
print(f"Size of {fname2} is {os.path.getsize(fname2)//1024} kbytes")

Size of image_ex2.tfrecord is 9768 kbytes


Notice: 

Storing image data as float gives a filesize of almost 10MB while the jpeg-encoded one was only 104kB. But now all preprocessing steps are complete, and the image can be fed directly to training (or possibly agumentation).

In [52]:
dataset2 = tf.data.TFRecordDataset(fname2)
img_example2 = next(iter(dataset2)) 
img_parsed2 = tf.train.Example.FromString(img_example2.numpy())
# extract features
xmin=img_parsed2.features.feature['image/object/bbox/xmin'].float_list.value[:]
xmax=img_parsed2.features.feature['image/object/bbox/xmax'].float_list.value[:]
ymin=img_parsed2.features.feature['image/object/bbox/ymin'].float_list.value[:]
ymax=img_parsed2.features.feature['image/object/bbox/ymax'].float_list.value[:]
by=img_parsed2.features.feature['image/by'].bytes_list.value[0].decode()
classes=img_parsed2.features.feature['image/object/class/text'].bytes_list.value[:]
class_label=img_parsed2.features.feature['image/object/class/label'].int64_list.value[:]
img_shape=img_parsed2.features.feature['image/shape'].int64_list.value[:]
img_data=img_parsed2.features.feature['image/data'].float_list.value[:]

In [ ]:
image2=np.array(img_data).reshape(img_shape) # reshape
image2=image2*255. # scale back to [0, 255] and convert to int
image2=image2.astype(int)
img=Image.fromarray(np.uint8(image2))   ###????
fig = plt.figure(figsize=(10,10))
axes = axes = fig.add_subplot(1, 1, 1)
plot_img(img, axes, xmin, ymin, xmax, ymax, classes, class_label, by)

Now, let's consider the case when we have a huge dataset, and need to create multiple TFRecords.

In [57]:
filelist=pd.read_pickle(pickles+'ArTaxOr_filelist.pkl')
filelist=filelist.sample(frac=1)
filelist.head()

,format,id,name,path,size,state,type
3454,jpg,b70685b172cd7203f8a8e298365b9de2,8d7f790a8d36.jpg,ArTaxOr/Hemiptera/8d7f790a8d36.jpg,"{'width': 2048, 'height': 1420}",2,1
8465,jpg,6de4966e24bdeb5c4ede4889dbdb776c,eabdc8f645a7.jpg,ArTaxOr/Diptera/eabdc8f645a7.jpg,"{'width': 936, 'height': 700}",2,1
2564,jpg,01363709e8efad370226af056c131428,2d88d964f109.jpg,ArTaxOr/Hemiptera/2d88d964f109.jpg,"{'width': 2048, 'height': 1536}",2,1
7540,jpg,c65fe155414fb17402badcc826d70eb8,79d124c047a8.jpg,ArTaxOr/Diptera/79d124c047a8.jpg,"{'width': 1536, 'height': 2048}",2,1
6155,jpg,2f28550a2d332e073605b572682ea40c,cd7acb747fb9.jpg,ArTaxOr/Coleoptera/cd7acb747fb9.jpg,"{'width': 1536, 'height': 2048}",2,1


#### The contextlib2 library
 is used to automatically close all TFRecords files after writing is finished.



In [ ]:
%%time
import contextlib2

def open_sharded_tfrecords(exit_stack, base_path, num_shards):
    tf_record_output_filenames = [
        '{}-{:05d}-of-{:05d}.tfrecord'.format(base_path, idx, num_shards)
                # f'{base_path}-{idx:05d}-of-{num_shards:05d}.tfrecord'.format(base_path, idx, num_shards)
        for idx in range(num_shards)
        ]
    print(base_path)
    tfrecords = [
        exit_stack.enter_context(tf.io.TFRecordWriter(file_name))
        for file_name in tf_record_output_filenames
    ]
    return tfrecords

num_shards=50
output_filebase='./ArTaxOr'

with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = open_sharded_tfrecords(tf_record_close_stack, output_filebase, num_shards)
    for i in range(len(filelist)):
        ldf=objectdf[objectdf.id == filelist.id.iloc[i]].reset_index()
        tf_record = create_tf_example(ldf, longest_edge=1280)
        output_shard_index = i % num_shards
        output_tfrecords[output_shard_index].write(tf_record.SerializeToString())

/content/ArTaxOr


In [ ]:
/content/ArTaxOr-00000-of-00050.tfrecord

# '{}-{:05d}-of-{:05d}.tfrecord'.format(base_path, idx, num_shards)
# f'{base_path}-{idx:05d}-of-{num_shards:05d}.tfrecord'.format(base_path, idx, num_shards)